In [2]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow.keras.backend as K
import numpy as np

In [ ]:


# --- Load patches ---
sar_patches = np.load("numpy_patches/sar_patches.npy")  # shape (7136,128,128)
mask_patches = np.load("numpy_patches/mask_patches_sar.npy")  # shape (7136,128,128)


s2_patches = np.load("C:/surfacewatermapping/Sen1Floods11/numpy_patches/s2_patches.npy")




In [5]:
mask_patches = np.expand_dims(mask_patches , axis= -1)
sar_patches = np.expand_dims(sar_patches , axis=-1)

print(s2_patches.shape)
print(mask_patches.shape)
print(sar_patches.shape)


(7136, 128, 128, 13)
(7136, 128, 128, 1)
(7136, 128, 128, 1)


In [6]:

# --- Dice coefficient & loss ---
def dice_coeff(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.BinaryCrossentropy()(y_true, y_pred)
    return bce + (1 - dice_coeff(y_true, y_pred))

# --- Your U-Net architecture ---
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    return x

def encoder_block(x, filters):
    c = conv_block(x, filters)
    p = layers.MaxPooling2D((2,2))(c)
    return c, p

def decoder_block(x, skip, filters):
    x = layers.Conv2DTranspose(filters, 2, strides=2, padding='same')(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

# --- Inputs ---
sar_input = layers.Input(shape=(128,128,1))
s2_input = layers.Input(shape=(128,128,13))

# --- Encoder ---
c1_sar, p1_sar = encoder_block(sar_input, 32)
c1_s2, p1_s2 = encoder_block(s2_input, 32)

c2_sar, p2_sar = encoder_block(p1_sar, 64)
c2_s2, p2_s2 = encoder_block(p1_s2, 64)

# --- Bottleneck ---
c3_sar, _ = encoder_block(p2_sar, 128)
c3_s2, _ = encoder_block(p2_s2, 128)
bottleneck = layers.Concatenate()([c3_sar, c3_s2])

# --- Decoder ---
d1 = decoder_block(bottleneck, layers.Concatenate()([c2_sar, c2_s2]), 128)
d2 = decoder_block(d1, layers.Concatenate()([c1_sar, c1_s2]), 64)

output = layers.Conv2D(1, 1, activation='sigmoid')(d2)

# --- Model ---
model = Model(inputs=[sar_input, s2_input], outputs=output)
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=[dice_coeff, 'accuracy'])
model.summary()

# --- Callbacks ---
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)




Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 13)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │      3,776 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │      9,248 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 64, 64,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 64, 64,    │     36,928 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 64, 64,    │     36,928 │ conv2d_6[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 32, 32,    │          0 │ conv2d_7[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 32, 32,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 32, 32,    │     73,856 │ max_pooling2d_3[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │    147,584 │ conv2d_8[0][0]  

 Total params: 1,293,761 (4.94 MB)

 Trainable params: 1,293,761 (4.94 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import gc

# --- Ensure float32 ---
sar_patches = sar_patches.astype('float32')
s2_patches = s2_patches.astype('float32')
mask_patches = mask_patches.astype('float32')

# --- Shuffle and split into train/val ---
num_samples = sar_patches.shape[0]
indices = np.arange(num_samples)
np.random.shuffle(indices)

split = int(num_samples * 0.9)  # 90% train, 10% val
train_idx, val_idx = indices[:split], indices[split:]

sar_train, sar_val = sar_patches[train_idx], sar_patches[val_idx]
s2_train, s2_val = s2_patches[train_idx], s2_patches[val_idx]
mask_train, mask_val = mask_patches[train_idx], mask_patches[val_idx]

# --- Delete original variables to free memory ---
del sar_patches, s2_patches, mask_patches
gc.collect()

# --- Create TensorFlow datasets ---
train_dataset = tf.data.Dataset.from_tensor_slices(
    ((sar_train, s2_train), mask_train)
).batch(16).shuffle(200)

val_dataset = tf.data.Dataset.from_tensor_slices(
    ((sar_val, s2_val), mask_val)
).batch(16)

# --- Callbacks ---
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_loss', save_best_only=True, mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# --- Train the model ---
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=12,
    verbose=1,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9448 - dice_coeff: 0.6410 - loss: 0.5223

402/402 ━━━━━━━━━━━━━━━━━━━━ 1345s 3s/step - accuracy: 0.9635 - dice_coeff: 0.7163 - loss: 0.4085 - val_accuracy: 0.9624 - val_dice_coeff: 0.6652 - val_loss: 0.5032
Epoch 2/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9696 - dice_coeff: 0.7534 - loss: 0.3539

402/402 ━━━━━━━━━━━━━━━━━━━━ 1337s 3s/step - accuracy: 0.9701 - dice_coeff: 0.7625 - loss: 0.3444 - val_accuracy: 0.9696 - val_dice_coeff: 0.7417 - val_loss: 0.3770
Epoch 3/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9718 - dice_coeff: 0.7706 - loss: 0.3286

402/402 ━━━━━━━━━━━━━━━━━━━━ 1635s 4s/step - accuracy: 0.9712 - dice_coeff: 0.7724 - loss: 0.3297 - val_accuracy: 0.9684 - val_dice_coeff: 0.7483 - val_loss: 0.3574
Epoch 4/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.9713 - dice_coeff: 0.7662 - loss: 0.3382

402/402 ━━━━━━━━━━━━━━━━━━━━ 1643s 4s/step - accuracy: 0.9720 - dice_coeff: 0.7796 - loss: 0.3217 - val_accuracy: 0.9708 - val_dice_coeff: 0.7545 - val_loss: 0.3513
Epoch 5/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9732 - dice_coeff: 0.7861 - loss: 0.3102

402/402 ━━━━━━━━━━━━━━━━━━━━ 1304s 3s/step - accuracy: 0.9725 - dice_coeff: 0.7830 - loss: 0.3159 - val_accuracy: 0.9707 - val_dice_coeff: 0.7611 - val_loss: 0.3399
Epoch 6/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.9730 - dice_coeff: 0.7916 - loss: 0.3033

402/402 ━━━━━━━━━━━━━━━━━━━━ 1417s 4s/step - accuracy: 0.9730 - dice_coeff: 0.7883 - loss: 0.3076 - val_accuracy: 0.9706 - val_dice_coeff: 0.7655 - val_loss: 0.3349
Epoch 7/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 1322s 3s/step - accuracy: 0.9732 - dice_coeff: 0.7889 - loss: 0.3062 - val_accuracy: 0.9714 - val_dice_coeff: 0.7658 - val_loss: 0.3462
Epoch 8/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 1212s 3s/step - accuracy: 0.9732 - dice_coeff: 0.7887 - loss: 0.3082 - val_accuracy: 0.9698 - val_dice_coeff: 0.7558 - val_loss: 0.3416
Epoch 9/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 1279s 3s/step - accuracy: 0.9734 - dice_coeff: 0.7909 - loss: 0.3047 - val_accuracy: 0.9709 - val_dice_coeff: 0.7685 - val_loss: 0.3370
Epoch 10/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 1312s 3s/step - accuracy: 0.9738 - dice_coeff: 0.7942 - loss: 0.2991 - val_accuracy: 0.9702 - val_dice_coeff: 0.7700 - val_loss: 0.3375
Epoch 11/12
402/402 ━━━━━━━━━━━━━━━━━━━━ 2112s 5s/step - accuracy: 0.9739 - dice_coeff: 0.7963 - loss: 0.2966 - val_accuracy: 0.97